<a href="https://www.kaggle.com/code/ahmedrushdi/sarcasm?scriptVersionId=126808432" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import numpy as np 
import pandas as pd

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/news-headlines-dataset-for-sarcasm-detection/Sarcasm_Headlines_Dataset_v2.json
/kaggle/input/news-headlines-dataset-for-sarcasm-detection/Sarcasm_Headlines_Dataset.json


In [2]:
import pandas as pd
import re
from nltk.tokenize import word_tokenize


json_file_path1 = '/kaggle/input/news-headlines-dataset-for-sarcasm-detection/Sarcasm_Headlines_Dataset.json'
json_file_path2 = '/kaggle/input/news-headlines-dataset-for-sarcasm-detection/Sarcasm_Headlines_Dataset_v2.json'


# Read the JSON files using pandas
data1 = pd.read_json(json_file_path1, lines=True)
data2 = pd.read_json(json_file_path2, lines=True)



# Combine the data into a single DataFrame
data = pd.concat([data1, data2])

data = data.drop('article_link', axis = 1).drop_duplicates()
data

,headline,is_sarcastic
0,former versace store clerk sues over secret 'b...,0
1,the 'roseanne' revival catches up to our thorn...,0
2,mom starting to fear son's web series closest ...,1
3,"boehner just wants wife to listen, not come up...",1
4,j.k. rowling wishes snape happy birthday in th...,0
...,...,...
28497,tyson holds contest to let fans submit new ide...,1
28509,increasingly cocky bernie sanders announces he...,1
28520,cash-strapped zuckerberg forced to sell 11 mil...,1
28544,grocery store bar actually has great little ha...,1


In [3]:
corpus = data['headline']
labels = data['is_sarcastic']


In [4]:
import tensorflow as tf
import gensim.downloader as api
import numpy as np
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from keras.layers import Embedding, LSTM, Dense, GRU, Dropout

In [5]:
w2v_model = api.load("glove-wiki-gigaword-50")

[==================================================] 100.0% 66.0/66.0MB downloaded


In [6]:
vocab_size = len(w2v_model.key_to_index) +2  # Add 1 for the padding token
embedding_dim = w2v_model.vector_size

embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, idx in w2v_model.key_to_index.items():
    embedding_matrix[idx+2] = w2v_model[word]

embedding_layer = Embedding(
    input_dim=vocab_size,
    output_dim=embedding_dim,
    weights=[embedding_matrix],
    trainable=False,
)

In [7]:
len(list(w2v_model.key_to_index.keys()))

400000

In [8]:
input_dim = vocab_size + 2  # Size of the vocabulary
input_length = 200  # Length of input sequences

text_vectorization = TextVectorization(
    max_tokens=input_dim,
    output_mode="int",
    output_sequence_length=input_length,
    vocabulary = list(w2v_model.key_to_index.keys())
)
# text_vectorization.set_vocabulary(['<UNK>']+list(w2v_model.key_to_index.keys()))

In [9]:
w2v_model['the'] - embedding_layer(text_vectorization([['the']]))[0,0]

<tf.Tensor: shape=(50,), dtype=float32, numpy=
array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
      dtype=float32)>

In [10]:
model = tf.keras.Sequential([
    text_vectorization,  # TextVectorization layer
    embedding_layer,  # Embedding layer
    LSTM(64, return_sequences = True, input_shape=(input_length, embedding_dim)), # LSTM layer 1
#     LSTM(64, return_sequences = True,), # LSTM layer 2
#     LSTM(64,), # LSTM layer 1
    Dropout(0.3),
    Dense(128, activation='sigmoid'),   # Output
    Dense(1, activation='sigmoid'),   # Output
])

In [11]:
# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(corpus, labels, epochs=20, batch_size=64, validation_split=0.2)

Epoch 1/20
357/357 [==============================] - 202s 524ms/step - loss: 0.6732 - accuracy: 0.5811 - val_loss: 0.7174 - val_accuracy: 0.4312
Epoch 2/20
357/357 [==============================] - 182s 510ms/step - loss: 0.6729 - accuracy: 0.5735 - val_loss: 0.7011 - val_accuracy: 0.3965
Epoch 3/20
357/357 [==============================] - 182s 509ms/step - loss: 0.6765 - accuracy: 0.5549 - val_loss: 0.7272 - val_accuracy: 0.3903
Epoch 4/20
357/357 [==============================] - 182s 510ms/step - loss: 0.6774 - accuracy: 0.5606 - val_loss: 0.7173 - val_accuracy: 0.3891
Epoch 5/20
357/357 [==============================] - 182s 511ms/step - loss: 0.6865 - accuracy: 0.5600 - val_loss: 0.7291 - val_accuracy: 0.3883
Epoch 6/20
357/357 [==============================] - 181s 507ms/step - loss: 0.6693 - accuracy: 0.6009 - val_loss: 0.6960 - val_accuracy: 0.4614
Epoch 7/20
357/357 [==============================] - 182s 509ms/step - loss: 0.6717 - accuracy: 0.5890 - val_loss: 0.7433 -

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVec  (None, 200)              0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 200, 50)           20000100  
                                                                 
 lstm (LSTM)                 (None, 200, 64)           29440     
                                                                 
 dropout (Dropout)           (None, 200, 64)           0         
                                                                 
 dense (Dense)               (None, 200, 128)          8320      
                                                                 
 dense_1 (Dense)             (None, 200, 1)            129       
                                                        